In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:

path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB051511,Karma Resorts,Other,Past Guests,Karma Bavaria,PF272488,Mrs,Cheryl,Carpenter,Female,...,2025-06-30 00:00:00,6,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
1,KB051512,Karma Resorts,Other,Past Guests,Karma Bavaria,PF272488,Mrs,Cheryl,Carpenter,Female,...,2025-06-30 00:00:00,6,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
2,KK076667,Karma Resorts,Other,Past Guests,Karma Kandara,PF244735,Miss,Bree,Harrison,Female,...,2025-07-04 00:00:00,7,HOUSE_USE,V_OWNER,COMP,2,5,0,Email and Phone,NO
3,KK075851,Karma Resorts,Other,Past Guests,Karma Kandara,PF318130,Miss,Kara,Wills,Female,...,2025-06-30 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
4,KK071860,Karma Resorts,Other,Past Guests,Karma Kandara,PF307896,Mr,Matthew,Creagh,Male,...,2025-07-04 00:00:00,4,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,KSM014597,Karma Resorts,Other,Past Guests,Karma St Martins,PF299892,<NA>,Terence,Turner,Unknown,...,2025-07-06 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
164,KB045480,Karma Resorts,Other,Past Guests,Karma Bavaria,PF291644,Mrs,Brenda,Nard,Female,...,2025-06-29 00:00:00,1,TA,TA,RETAIL,2,0,0,Email and Phone,NO
165,KK075144,Karma Resorts,Other,Past Guests,Karma Kandara,PF316423,Ms,Liliana O,Barston,Female,...,2025-07-04 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
166,KK076458,Karma Resorts,Other,Past Guests,Karma Kandara,PF319749,Miss,Katia,Van+Roosbroeck,Female,...,2025-07-01 00:00:00,3,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [8]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
1,KB051512,Karma Resorts,Other,Past Guests,Karma Bavaria,PF272488,Mrs,Cheryl,Carpenter,Female,...,2025-06-30 00:00:00,6,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
5,KK071962,Karma Resorts,Other,Past Guests,Karma Kandara,PF307896,Mr.,Matthew James,Creagh,Male,...,2025-07-04 00:00:00,4,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
12,KK074792,Karma Resorts,Other,Past Guests,Karma Kandara,PF314394,Miss,Shuchen,Li,Female,...,2025-07-01 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
20,KB051415,Karma Resorts,Other,Past Guests,Karma Bavaria,PF231603,Mr,Andreas,Kowolik,Unknown,...,2025-07-04 00:00:00,4,COMP,GER_OWE,COMP,1,0,0,Email Only,NO
25,KB050517,Karma Resorts,Other,Past Guests,Karma Bavaria,PF178352,Mrs,Wei,Yu,Female,...,2025-07-06 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
30,KB051443,Karma Resorts,Other,Past Guests,Karma Bavaria,PF250879,Mr,Georg,Freundorfer,Male,...,2025-07-01 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
41,KB047179,Karma Resorts,Other,Past Guests,Karma Bavaria,PF300180,Frau,Heike,Demel,Female,...,2025-07-05 23:00:00,0,LGR,TEL,RETAIL,1,0,0,Email and Phone,NO
57,KB043407,Karma Resorts,Other,Past Guests,Karma Bavaria,PF280089,Mrs,Julia,Dressen,Female,...,2025-06-29 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
58,KB043410,Karma Resorts,Other,Past Guests,Karma Bavaria,PF280089,Mrs,Julia,Dressen,Female,...,2025-06-29 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
61,KLM010333,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF227600,Mr,Test Daniel,Richards,Male,...,2025-07-02 00:00:00,1,COMP,STAFF,COMP,2,0,0,Email Only,NO


In [9]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB051511,Karma Resorts,Other,Past Guests,Karma Bavaria,PF272488,Mrs,Cheryl,Carpenter,Female,...,2025-06-30 00:00:00,6,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
1,KK076667,Karma Resorts,Other,Past Guests,Karma Kandara,PF244735,Miss,Bree,Harrison,Female,...,2025-07-04 00:00:00,7,HOUSE_USE,V_OWNER,COMP,2,5,0,Email and Phone,NO
2,KK075851,Karma Resorts,Other,Past Guests,Karma Kandara,PF318130,Miss,Kara,Wills,Female,...,2025-06-30 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
3,KK071860,Karma Resorts,Other,Past Guests,Karma Kandara,PF307896,Mr,Matthew,Creagh,Male,...,2025-07-04 00:00:00,4,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
4,KK076575,Karma Resorts,Other,Past Guests,Karma Kandara,PF320328,Mrs,Samantha,Lawrence,Female,...,2025-07-01 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,5,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,KSM014597,Karma Resorts,Other,Past Guests,Karma St Martins,PF299892,empty,Terence,Turner,Unknown,...,2025-07-06 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
139,KB045480,Karma Resorts,Other,Past Guests,Karma Bavaria,PF291644,Mrs,Brenda,Nard,Female,...,2025-06-29 00:00:00,1,TA,TA,RETAIL,2,0,0,Email and Phone,NO
140,KK075144,Karma Resorts,Other,Past Guests,Karma Kandara,PF316423,Ms,Liliana O,Barston,Female,...,2025-07-04 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
141,KK076458,Karma Resorts,Other,Past Guests,Karma Kandara,PF319749,Miss,Katia,Van+Roosbroeck,Female,...,2025-07-01 00:00:00,3,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO


In [10]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [11]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [12]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
1,KK076667,Karma Resorts,Other,Past Guests,Karma Kandara,PF244735,Miss,Bree,Harrison,Female,...,2025-07-04 00:00:00,7,HOUSE_USE,V_OWNER,COMP,2,5,0,Email and Phone,NO
13,KB051336,Karma Resorts,Other,Past Guests,Karma Bavaria,PF242189,Frau,Sabine,Rimschneider,Female,...,2025-07-01 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
16,KB051414,Karma Resorts,Other,Past Guests,Karma Bavaria,PF319019,Mr,Daniel,Kowolik,Male,...,2025-07-04 00:00:00,4,COMP,GER_OWE,COMP,1,0,0,Email Only,NO
17,KB051052,Karma Resorts,Other,Past Guests,Karma Bavaria,PF264596,Mr,Uwe,Grassl,Male,...,2025-06-30 00:00:00,1,COMP,GER_OWE,COMP,1,0,0,Email Only,NO
19,KB051298,Karma Resorts,Other,Past Guests,Karma Bavaria,PF118055,Mr,Tahir,Karim,Male,...,2025-06-30 00:00:00,3,COMP,GER_OWE,COMP,4,0,0,Email and Phone,NO
20,KB051340,Karma Resorts,Other,Past Guests,Karma Bavaria,PF178352,Mrs,Wei,Yu,Female,...,2025-07-01 00:00:00,1,COMP,GER_OWE,COMP,1,0,0,Email and Phone,NO
21,KB048039,Karma Resorts,Other,Past Guests,Karma Bavaria,PF168612,Herr,Michael,Caspers,Male,...,2025-07-06 00:00:00,6,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
24,KB051441,Karma Resorts,Other,Past Guests,Karma Bavaria,PF250879,Mr,Georg,Freundorfer,Male,...,2025-07-01 00:00:00,1,COMP,GER_OWE,COMP,1,0,0,Email and Phone,NO
26,KB051448,Karma Resorts,Other,Past Guests,Karma Bavaria,PF192683,Mrs,Kathrin,Haas,Female,...,2025-07-01 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
27,KB051577,Karma Resorts,Other,Past Guests,Karma Bavaria,PF304831,empty,Alexandra,Babic,Female,...,2025-07-01 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO


In [13]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [14]:
count_data_akhir = df_analytic.shape[0]

In [15]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB044610,Karma Resorts,Other,Past Guests,Karma Bavaria,PF287202,Herr,Roman,Handschin,Unknown,...,2025-06-29 00:00:00,2,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
1,KLM007285,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF292899,Ms,Helen,Kite,Female,...,2025-06-29 00:00:00,2,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
2,KLM007291,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF292908,Ms,Hannah,Blaikie,Female,...,2025-06-29 00:00:00,2,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
3,KLM007293,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF292911,Mr,Andrew,Morrison,Male,...,2025-06-29 00:00:00,2,WEDDING,WEDDING,GROUP,2,0,0,Email and Phone,NO
4,KLM007279,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF292888,Mr,James,Ferguson,Male,...,2025-06-29 00:00:00,2,WEDDING,WEDDING,GROUP,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,KB050086,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312063,empty,Ann-Katrin,Kaufhold,Unknown,...,2025-07-06 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
122,KB045459,Karma Resorts,Other,Past Guests,Karma Bavaria,PF291554,empty,Deborah,Koltzsch,Unknown,...,2025-07-06 00:00:00,1,OTA,OTA,RETAIL,7,0,0,Phone Only,NO
123,KB048552,Karma Resorts,Other,Past Guests,Karma Bavaria,PF306146,empty,Katharina,Harlos,Unknown,...,2025-07-06 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
124,KSM014871,Karma Resorts,Other,Past Guests,Karma St Martins,PF305987,empty,Marian,Bampoe,Unknown,...,2025-07-07 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [16]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [17]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_16952\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [18]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB044610,Karma Resorts,Other,Past Guests,Karma Bavaria,PF287202,Herr,Roman,Handschin,Unknown,...,2025-06-29,2,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
1,KLM007292,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF292910,Ms,Rosie,Laidlaw,Female,...,2025-06-29,2,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
2,KSM015587,Karma Resorts,Other,Past Guests,Karma St Martins,PF314498,Mr,Peter,Larkin,Male,...,2025-06-29,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KSH018898,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF277604,Mrs,Odette,Williams,Female,...,2025-06-29,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
4,KLM007286,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF292901,Ms,Nicole,Easton,Female,...,2025-06-29,2,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,KB050086,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312063,,Ann-Katrin,Kaufhold,Unknown,...,2025-07-06,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
122,KB045459,Karma Resorts,Other,Past Guests,Karma Bavaria,PF291554,,Deborah,Koltzsch,Unknown,...,2025-07-06,1,OTA,OTA,RETAIL,7,0,0,Phone Only,NO
123,KB048552,Karma Resorts,Other,Past Guests,Karma Bavaria,PF306146,,Katharina,Harlos,Unknown,...,2025-07-06,1,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
124,KSM014871,Karma Resorts,Other,Past Guests,Karma St Martins,PF305987,,Marian,Bampoe,Unknown,...,2025-07-07,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [19]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 168
Jumlah Data Yang Duplicate : 25
Jumlah Data Setelah Hapus Duplicate : 143
Jumlah Data Yang Termasuk Member: 17
Jumlah Data Setelah Hapus Member : 126
Jumlah Data Akhir : 126


In [20]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)